# Sensitivity Analysis

## 1. Define the problem and policies

In [3]:
# Imports

import numpy as np
import pandas as pd
import time

import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('white')

from ema_workbench import (Model, MultiprocessingEvaluator, perform_experiments, ema_logging, Policy, Scenario)
ema_logging.log_to_stderr(ema_logging.INFO)

from ema_workbench.em_framework.evaluators import LHS, SOBOL, MORRIS

from ema_workbench.analysis import feature_scoring
from ema_workbench.analysis.scenario_discovery_util import RuleInductionType
from ema_workbench.em_framework.salib_samplers import get_SALib_problem
from ema_workbench.em_framework.samplers import sample_uncertainties
from ema_workbench.util import ema_logging, save_results, load_results


from SALib.analyze import sobol


C:\Users\frac1\Anaconda3\lib\site-packages\ema_workbench\em_framework\evaluators.py:22: UserWarning: ipyparallel not installed - IpyparalleEvaluator not available
  'ipyparallel not installed - IpyparalleEvaluator not available')


In [4]:
# Specification of the problem

from problem_formulation import get_model_for_problem_formulation
dike_model, planning_steps = get_model_for_problem_formulation(1)

# Specification of the policies

policy_noaction = Policy("NoAction",  
                    **{'0_RfR 0': 0, '0_RfR 1' : 0, '0_RfR 2' : 0,
                    '1_RfR 0': 0, '1_RfR 1' : 0, '1_RfR 2' : 0,
                    '2_RfR 0': 0, '2_RfR 1' : 0, '2_RfR 2' : 0,
                    '3_RfR 0': 0, '3_RfR 1' : 0, '3_RfR 2' : 0,
                    '4_RfR 0': 0, '4_RfR 1' : 0, '4_RfR 2' : 0,
                    'A.1_DikeIncrease 0' : 0, 'A.1_DikeIncrease 1' : 0, 'A.1_DikeIncrease 2' : 0,
                    'A.2_DikeIncrease 0' : 0, 'A.2_DikeIncrease 1' : 0, 'A.2_DikeIncrease 2' : 0,
                    'A.3_DikeIncrease 0' : 0, 'A.3_DikeIncrease 1' : 0, 'A.3_DikeIncrease 2' : 0,
                    'A.4_DikeIncrease 0' : 0, 'A.4_DikeIncrease 1' : 0, 'A.4_DikeIncrease 2' : 0,
                    'A.5_DikeIncrease 0' : 0, 'A.5_DikeIncrease 1' : 0, 'A.5_DikeIncrease 2' : 0,
                    'EWS_DaysToThreat':  0   })

## 2. Sobol's scores

### 2.1 Execute Sensistivity Analysis on uncertainties

We will run a total of N(2K+2) = 10(2\*19+2) experiments, since the dike model has K=19 uncertainties and we use a baseline number of experiments equalling N=10

In [6]:
len(dike_model.uncertainties)

19

In [3]:
# Convert the problem into an SA-equivalent problem

sa_problem = get_SALib_problem(dike_model.uncertainties)

In [8]:
# Execute Sensistivity Analysis over uncertainties

baseline_n_experiments = 10
start = time.time()

with MultiprocessingEvaluator(dike_model) as evaluator:
     experiments, outcomes = evaluator.perform_experiments(scenarios = baseline_n_experiments,
                                                           policies = [policy_noaction], 
                                                           uncertainty_sampling = SOBOL)
end = time.time()
print('Sobol time is ' + str(round((end - start)/60)) + ' minutes')

[MainProcess/INFO] pool started
[MainProcess/INFO] performing 400 scenarios * 1 policies * 1 model(s) = 400 experiments
[MainProcess/INFO] 40 cases completed
[MainProcess/INFO] 80 cases completed
[MainProcess/INFO] 120 cases completed
[MainProcess/INFO] 160 cases completed
[MainProcess/INFO] 200 cases completed
[MainProcess/INFO] 240 cases completed
[MainProcess/INFO] 280 cases completed
[MainProcess/INFO] 320 cases completed
[MainProcess/INFO] 360 cases completed
[MainProcess/INFO] 400 cases completed
[MainProcess/INFO] experiments finished
[MainProcess/INFO] terminating pool


Sobol time is 5 mintues


In [10]:
# save the results

u_results = (experiments, outcomes)
save_results(u_results, 'Sobol_results_uncertainties_10ex_NoActionpol.tar.gz')

[MainProcess/INFO] results saved successfully to C:\Users\frac1\Documents\GitHub\Model Based DM - Assignment collaboration\model-based-decision-making\model\Sobol_results_uncertainties_200sc_NoActionpol.tar.gz


### 2.2 Execute Sensistivity Analysis on levers

We will run a sensitivity analysis over a total of N(2K+2) = 10(2\*31+2) experiments, since the dike model has K=31 levers and we use a baseline number of experiments equalling N=10 under the reference scenario (set as the worst-case scenario, and found from the notebook 1 - Reference Case Scenario)

In [5]:
len(dike_model.levers)

31

In [6]:
# Convert the problem into an SA-equivalent problem

sa_problem = get_SALib_problem(dike_model.levers)

In [7]:
# Load policy results of a NoAction policy under 500 scenarios (notebook 1-Reference Case Scenario)
[experiments, outcomes] = load_results("../results/500Scenarios_NoAction_PF1.tar.gz")
outcomes_df = pd.DataFrame(data=outcomes)

#Get index of worst-case scenario
index_wc = outcomes_df.sort_values("Expected Number of Deaths").tail(1).index
experiment_wc = experiments.iloc[index_wc]

#Set the reference scenario as the worst-case scenario
reference_scenarios = [Scenario(f"{index}", **row) for index, row in experiment_wc.iloc[0:,0:19].iterrows()]

[MainProcess/INFO] results loaded succesfully from C:\Users\frac1\Documents\GitHub\Model Based DM - Assignment collaboration\model-based-decision-making\results\500Scenarios_NoAction_PF1.tar.gz


In [8]:
# Execute Sensistivity Analysis over levers

baseline_n_experiments = 10

start = time.time()
with MultiprocessingEvaluator(dike_model) as evaluator:
     experiments, outcomes = evaluator.perform_experiments(scenarios = reference_scenarios,
                                                           policies = baseline_n_experiments,
                                                           levers_sampling = SOBOL)
end = time.time()
print('Sobol time is ' + str(round((end - start)/60)) + ' minutes')

[MainProcess/INFO] pool started
[MainProcess/INFO] performing 1 scenarios * 640 policies * 1 model(s) = 640 experiments
[MainProcess/INFO] 64 cases completed
[MainProcess/INFO] 128 cases completed
[MainProcess/INFO] 192 cases completed
[MainProcess/INFO] 256 cases completed
[MainProcess/INFO] 320 cases completed
[MainProcess/INFO] 384 cases completed
[MainProcess/INFO] 448 cases completed
[MainProcess/INFO] 512 cases completed
[MainProcess/INFO] 576 cases completed
[MainProcess/INFO] 640 cases completed
[MainProcess/INFO] experiments finished
[MainProcess/INFO] terminating pool


Sobol time is 8 mintues


In [10]:
#save the results

l_results = (experiments, outcomes)
save_results(l_results, 'Sobol_results_levers_10ex_RefScenario.tar.gz')

[MainProcess/INFO] results saved successfully to C:\Users\frac1\Documents\GitHub\Model Based DM - Assignment collaboration\model-based-decision-making\model\Sobol_results_levers_10ex_RefScenario.tar.gz
